In [1]:
class Simulation:
    def __init__(self, parameters):
        self.parameters = parameters
        self.population_species_history_1 = [parameters.initial_population_species_1]
        self.population_species_history_2 = [parameters.initial_population_species_2]
        self.z_m1_history = [parameters.z_m1_initial]
        self.z_m2_history = [parameters.z_m2_initial]
        self.z_m1 = parameters.z_m1_initial
        self.z_m2 = parameters.z_m2_initial

    def next_generation_ecology(self):
        N1 = self.population_species_history_1[-1]
        N2 = self.population_species_history_2[-1]
      
    
        if N1 > 0.01:

            next_generation_competetion_term_sp1 = (self.parameters.b1 * N1) / (1 + self.parameters.alpha11 * N1 + self.parameters.alpha12 * N2)
            next_generation_reproductive_interference_term_sp1 = (
                    N1 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1 * self.z_m1)
                ) / (
                    N1 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1 * self.z_m1) + 
                    N2 * self.parameters.mean_heterospecifc_permissivness_females_sp_1 * (1 - self.z_m2)
                )
            next_N1 = next_generation_competetion_term_sp1*next_generation_reproductive_interference_term_sp1
            next_N1 = max(next_N1, 0)
        else:
            next_N1 = 0

        if N2 > 0.01:

            next_generation_competetion_term_sp2 = (self.parameters.b2 * N2) / (
                1 + self.parameters.alpha22 * N2 + self.parameters.alpha21 * N1
            )

            next_generation_reproductive_interference_term_sp2 = (
                N2 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2 * self.z_m2)
            ) / (
                N2 * (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2 * self.z_m2) + 
                N1 * self.parameters.mean_heterospecifc_permissivness_females_sp_2 * (1 - self.z_m1)
            )

            next_N2 = next_generation_competetion_term_sp2 * next_generation_reproductive_interference_term_sp2

            next_N2 = max(next_N2, 0)
        else:
            next_N2 = 0

        self.population_species_history_1.append(next_N1)
        self.population_species_history_2.append(next_N2)

    def next_generation_traits(self, N1, N2):
        if N1 > 0.01:
            delta_z_m1 = self.parameters.V_A1 * (self.parameters.conspecfic_mating_rate_multiplier_sp_1) / (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_1*self.z_m1)
            new_z_m1 = self.z_m1 + delta_z_m1
            
            if new_z_m1 <= 1:
                self.z_m1 = new_z_m1
            else:
                self.z_m1 = 1

            self.z_m1_history.append(self.z_m1)

        if N2 > 0.01:
            delta_z_m2 = self.parameters.V_A2 * (self.parameters.conspecfic_mating_rate_multiplier_sp_2) / (1 + self.parameters.conspecfic_mating_rate_multiplier_sp_2*self.z_m2)

            new_z_m2 = self.z_m2 + delta_z_m2
            
            if new_z_m2 <= 1:
                self.z_m2 = new_z_m2
            else:
                self.z_m2 = 1

            self.z_m2_history.append(self.z_m2)
            
    def next_generation(self):
        self.next_generation_ecology()
        N1 = self.population_species_history_1[-1]
        N2 = self.population_species_history_2[-1]
        self.next_generation_traits(N1, N2)
        return N1, N2

    def run_simulation(self):
        for t in range(self.parameters.num_generations - 1):
            N1, N2 = self.next_generation()
            if N1 < 0.01 or N2 < 0.01:
                break

        return self.population_species_history_1, self.population_species_history_2, self.z_m1_history, self.z_m2_history